# Reads

In [ ]:
https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right

a figura com drvs justifica muito bem a escolha de 80% para teste


fazer eval em todos dfs de 80%, adicionar epoch e indices etc para depois pegar caso necessario
pegar melhor modelo olhando o loss e aplicar pra todos circuitos de teste fcn, ou fcn_large



Figura:
como alguem faria para usar o preditor
como alguem faria para testar com outra tecnologia o preditor
explicar que ele he independente de tecnologia
fluxograma mostrando a parte do ORDF e a parte do INNOVUS para geracao dos benchamrks
dps para geracao dos dados de treino e inferencia.
Explicar o problema dos dados, não é só uma questão de armazenamento, mas a leitura deles
tbm seria muito demorada inviabilizando o treino do modelo.
justificar o porque de pegar soh short

# Imports

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import shutil
import os
import time

# Training Result

In [ ]:
# Training performace
trainResultDF = pickle.load(open('results/fcn/trainResultDF.pkl', 'rb'))
epochs = {x for x in trainResultDF['epoch']}
print('epoch, tp, tn, fp, fn')
for x in range(len(epochs)):
  epochDF = trainResultDF.loc[trainResultDF['epoch'] == x]
  print(x, epochDF['tp'].sum(), epochDF['tn'].sum(), epochDF['fp'].sum(), epochDF['fn'].sum(), sep=',')

In [ ]:
# Plot training metrics for a given design
# TODO: Maybe change this to take the average for all types of the same design
trainResultDF = pickle.load(open('results/fcn/trainResultDF.pkl', 'rb'))
design = 'aes_81'
sortedDF = trainResultDF.loc[trainResultDF['design'] == design].sort_values(by=['epoch'])
ytrain = [x for x in sortedDF['tp']]
yval = [x for x in sortedDF['tn']]
plt.plot(ytrain, label = "tp")
plt.plot(yval, label = "tn")
plt.legend()
plt.title(design)
plt.show()

# Cross Validation Result

In [ ]:
crossValPath = '/home/sheiny/workspace/Predictor/results/'
pkls = [crossValPath+x for x in os.listdir(crossValPath) if '.pkl' in x]

cvDicts = []
for pkl in pkls:
  df = pickle.load(open(pkl, 'rb'))
  runs = {(row['Design'], row['Density']) for index, row in df.iterrows()}
  runLabel = ''
  for x in runs:
    runLabel += x[0]+str(x[1])+', '
  tp = sum(df['tp'])
  tn = sum(df['tn'])
  fp = sum(df['fp'])
  fn = sum(df['fn'])
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  TotalPos = tp+fn
  TotalNeg = fp+tn
  PosRatio = ((tp+fn)/(tp+fn+fp+tn))*100
  tpr = tp/(tp+fn)
  spc = tn/(tn+fp)
  acc = (tp+tn)/(tp+tn+fp+fn)
  sqrt = math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
  mcc = 0
  if sqrt != 0:
    mcc = ((tp * tn) - (fp * fn))/sqrt
  cvDicts.append({'Designs':runLabel[:-2],
                  'tp':tp, 'tn':tn, 'fp':fp, 'fn':fn,
                  'PosRatio%':PosRatio,
                  'tpr':tpr, 'spc':spc, 'acc':acc, 'MCC[-1:1]':mcc})
cvDf = pd.DataFrame.from_dict(cvDicts)
cvDf.set_index('Designs', inplace=True)
cvDf.loc['mean'] = cvDf.mean()
cvDf.loc['min'] = cvDf.min()
cvDf.loc['max'] = cvDf.max()
cvDf.to_csv('results/cvResult.csv')

# Prediction Results

In [ ]:
pkl = '/home/sheiny/workspace/Predictor/results/predictRuns.pkl'
df = pickle.load(open(pkl, 'rb'))

circuitResultDict = []
runs = {(row['Design'], row['Density']) for index, row in df.iterrows()}
for run in runs:
  tempDf = df.loc[(df['Design'] == run[0]) & (df['Density'] == run[1])]
  tp = sum(tempDf['tp'])
  tn = sum(tempDf['tn'])
  fp = sum(tempDf['fp'])
  fn = sum(tempDf['fn'])
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  TotalPos = tp+fn
  TotalNeg = fp+tn
  PosRatio = ((tp+fn)/(tp+fn+fp+tn))*100
  tpr = tp/(tp+fn)
  spc = tn/(tn+fp)
  acc = (tp+tn)/(tp+tn+fp+fn)
  sqrt = math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
  mcc = 0
  if sqrt != 0:
    mcc = ((tp * tn) - (fp * fn))/sqrt
  circuitResultDict.append({'Design':run[0]+str(run[1]),
                            'tp':str(tp)+' ('+str(int(tpr*100))+'%)', 'tn':int(tn),
                            'fp':int(fp), 'fn':int(fn),
                            'PosRatio%':PosRatio,
                            'spc %':spc*100, 'acc %':acc*100, 'MCC[-1:1]':mcc})
resultDf = pd.DataFrame.from_dict(circuitResultDict)
resultDf.set_index('Design', inplace=True)
resultDf.to_csv('results/predictionResult.csv')

# Plot DRVs and Predictions

In [ ]:
design = '/data/CSVWhole/aes/'
modelPath = 'results/fcn/model_18.pkl'
outPath = 'predictions/aes18'



def writePredictionFile(outFile, nodesToWrite):
  with open(outFile, 'w') as fp:
    for node in nodesToWrite:
      fp.write(str(node)+'\n')
  fp.close()

def predict(model, pkl):
  testDf = pd.read_pickle(pkl, compression='zip')
  labels = testDf.pop(testDf.columns.values[-1])
  nodeIDs = testDf.pop(testDf.columns.values[0])#drop first column which contains the nodeIds
  testHyperImages = np.array(testDf).reshape(len(testDf),22,33,33)
  predictions = model.predict(testHyperImages)
  return nodeIDs, predictions, labels

model = pickle.load(open(modelPath, 'rb'))
pkls = [design+x for x in os.listdir(design) if '.pkl' in x]
pkls.sort(key = lambda x : int(x[x.rfind('_')+1:x.find('.')]))
results = []
for pkl in pkls:
  results.append(predict(model, pkl))

tp = []
tn = []
fp = []
fn = []
for x, y, z in results:
  nodeIDs = [int(n) for n in x]
  predictions = y > 0.5
  for id, prediction, actual in zip(nodeIDs, predictions, z):
    if prediction == actual and actual == True:
      tp.append(id)
    elif prediction == actual and actual == False:
      tn.append(id)
    elif prediction != actual and actual == True:
      fn.append(id)
    else:
      fp.append(id)

writePredictionFile(outPath+'tp.txt', tp)
writePredictionFile(outPath+'tn.txt', tn)
writePredictionFile(outPath+'fp.txt', fp)
writePredictionFile(outPath+'fn.txt', fn)

# Benchmark Information Plot

In [ ]:
# Beware the number of components is wrong!!! It is considering filler cells
df = pd.read_pickle('benchmarkInfo/benchmark.pkl', compression='zip')

In [ ]:
# Benchmark Info
benchmarkDf = df.loc[df['Design'].str.contains('80')]
columnsToDrop = ['SPECIALNETS','IDRShort', 'IDRTotal', 'FDRShort', 'FDRTotal', 'GR', 'IDR', 'FDR']
benchmarkDf = benchmarkDf.drop(columns=columnsToDrop)
benchmarkDf.sort_values(by=['COMPONENTS'], ascending=False)
benchmarkDf.to_csv('benchmarkDf.csv', index=False)

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
numIDRShorts = dfRuntime['IDRShort']
dfRuntime = dfRuntime[['GR', 'IDR', 'FDR']]
ax = dfRuntime.plot.bar(stacked=True)

numBars = int(len(ax.patches)/3)
for rect, value in zip(ax.patches[numBars:numBars*2], numIDRShorts):
  h = rect.get_height() /2.
  w = rect.get_width() /2.
  x, y = rect.get_xy()
  ax.text(x+w, y+h, value, horizontalalignment='center',verticalalignment='center')

ax.set_ylabel('Runtime in seconds')
ax.set_xlabel('Designs (80% row utilization)')
ax.legend(["Global Routing", "Initial Detailed Routing", "Complete Detailed Routing"])
plt.title('Routing Runtime Break Down')
# plt.savefig('routing_runtime.pdf')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
idrShort = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  avgIDRShort = sum(tempDf['IDRShort']/len(tempDf))
  idrShort.append(avgIDRShort)
    
plt.plot([y for y in range(70, 91)], idrShort, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Initial Detailed Routing Short Violations (IDRV)")
plt.title('Average IDR Short x Row Utilization (Only fully routable circuits)')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
fdrRuntime = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  fdr = sum(tempDf['FDR']/len(tempDf))
  fdrRuntime.append(fdr)

plt.plot([y for y in range(70, 91)], fdrRuntime, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Runtime (seconds)")
plt.title('Average Runtime to complete routing (Only fully routable circuits)')